In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as img
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.pooling import MaxPool2d
from glob import glob
import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
train_df = pd.read_csv("../input/data/train/preprocessing_data.csv")
train_df = train_df.sort_values("id")

In [4]:
transform = transforms.Compose([
    transforms.Resize((512, 384), Image.BILINEAR),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.5,0.5,0.5,0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

In [5]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=False)

train_split = []
test_split = []
for train_index, test_index in kf.split(train_df):
    train,test = train_df[["image_path","label"]].loc[train_index], train_df[["image_path","label"]].loc[test_index]
#     y_train, y_test, = train_df["label"].loc[train_index], train_df["label"].loc[test_index]
    train_split.append(train)
    test_split.append(test)


In [6]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform):
        self.X = dataframe["image_path"]
        self.y = dataframe["label"]
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.X.iloc[index])
        label = self.y.iloc[index]

        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

    def __len__(self):
        return len(self.X)

In [7]:
train_data = CustomDataset(train,transform)
eval_data = CustomDataset(test,transform)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=1)
eval_dataloader = torch.utils.data.DataLoader(eval_data, batch_size=128, shuffle=True, num_workers=1)

In [38]:
class VGG(nn.Module):
    def __init__(self, num_classes=18,init_weights=True):
        super(VGG, self).__init__()
        
            
        self.convlayers = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2), # 112
            
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2), # 56
            
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2), # 28
            
            
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2), # 14
            
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2), # 7
        )
        if init_weights:
            self._initialize_weights()
            
        self.fclayer = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096,num_classes)
        )
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


    def forward(self, train):
        output = self.convlayers(train)
        output = torch.flatten(output,1)
        output = self.fclayer(output)
        return output

In [57]:
vggnet = VGG(18).to(device)

In [58]:
# Loss
criterion = nn.CrossEntropyLoss()

# optimizer 
# optimizer = optim.Adam(vggnet.parameters(),lr=0.00001)
optimizer = optim.SGD(vggnet.parameters(),lr=0.01,momentum=0.9,weight_decay=0.0005)
from torch.optim import lr_scheduler
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma = 0.1)

In [59]:
dataloaders = {
    "train" : train_dataloader,
    "test" : eval_dataloader
}

In [60]:
import time
import copy
from tqdm.notebook import tqdm

def train_model(model, dataloaders,criterion,optimizer,scheduler,num_epochs):
    start_time = time.time()
    max_accuracy = 0
    min_loss = int(1e9)
    early_stop_point = 0 
    early_stop_limit = 2
    stop_epoch = True
    best_model_wts = copy.deepcopy(model.state_dict())
    history = {'train_loss': [],"test_loss" : [], "train_acc":[],'test_acc':[]}
    
    for epoch in range(num_epochs):
        for phase in ['train', 'test']:
                if phase == 'train':
                    model.train()  # 모델을 학습 모드로 설정
                else:
                    model.eval()   # 모델을 평가 모드로 설정

                running_loss = 0.0
                running_acc = 0.0
                
                # 데이터 반복
                for step, (inputs, labels) in enumerate(tqdm(dataloaders[phase])):
                    inputs, labels = inputs.to(device), labels.to(device)

                    # 매개변수 경사도 0으로 초기화
                    optimizer.zero_grad()

                    # 순전파
                    # train에서만 연산 기록 추적
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)
        
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                        
                    running_loss += loss.item() * inputs.size(0)
                    running_acc += torch.sum(preds == labels.data)
                if phase == "train":
                    scheduler.step()

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_acc / len(dataloaders[phase].dataset)
                
                
                if phase == "train":
                    history["train_loss"].append(epoch_loss)
                    history["train_acc"].append(epoch_acc)
                else:
                    history["test_loss"].append(epoch_loss)
                    history["train_acc"].append(epoch_loss)
                    
                print(f"[Epoch : %d] [Phase : %s] epoch_loss : %.5f epoch_acc: %.2f" % 
                      (epoch+1, phase, epoch_loss, epoch_acc))
                
                
                            
                if phase == "test" and max_accuracy < epoch_acc:
                    max_accuracy = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    early_stop_point = 0
#                     for g in optimizer.param_groups:
#                             g['lr']/=10
                    
                    
                elif phase == "test" and max_accuracy > epoch_acc:
                    early_stop_point += 1
                    
                if phase == "test" and min_loss > epoch_loss:
                    min_loss = epoch_loss
                    
                if early_stop_point == early_stop_limit:
                    print("early_stopped")
                    stop_epoch = False
                    break
                    
                
        if stop_epoch == False:
            break
                
    print(time.time()-start_time)
    model.load_state_dict(best_model_wts)
    print("Done")
    return model

In [61]:
model_ft = train_model(vggnet, dataloaders,criterion, optimizer, lr_scheduler,num_epochs=100)


[Epoch : 1] [Phase : train] epoch_loss : 2.35726 epoch_acc: 0.26



[Epoch : 1] [Phase : test] epoch_loss : 2.09762 epoch_acc: 0.37



[Epoch : 2] [Phase : train] epoch_loss : 1.90661 epoch_acc: 0.40



[Epoch : 2] [Phase : test] epoch_loss : 1.67587 epoch_acc: 0.46



[Epoch : 3] [Phase : train] epoch_loss : 1.51076 epoch_acc: 0.51


...


[Epoch : 8] [Phase : test] epoch_loss : 0.68659 epoch_acc: 0.78



[Epoch : 9] [Phase : train] epoch_loss : 0.46603 epoch_acc: 0.84



[Epoch : 9] [Phase : test] epoch_loss : 0.76470 epoch_acc: 0.77



[Epoch : 10] [Phase : train] epoch_loss : 0.41411 epoch_acc: 0.86



[Epoch : 10] [Phase : test] epoch_loss : 1.07755 epoch_acc: 0.72
early_stopped
2091.4013710021973
Done


In [62]:
valid_dataloader = DataLoader(eval_data,shuffle=False)
def wrong_data(model):
    wrong_pred = []

    with torch.no_grad():
        model.eval()
        for i, (inputs, labels) in enumerate(valid_dataloader):
            inputs = inputs.to(device)
            outputs = model_ft(inputs)
            _, preds = torch.max(outputs, 1)
            
            
            wrong_pred.append([test.iloc[i]["image_path"],labels.cpu().numpy()[0],preds.cpu().numpy()[0]])
    wrong_pred = pd.DataFrame(wrong_pred, columns=["image_path","label","pred"])
    return wrong_pred

In [63]:
ee = wrong_data(model_ft)

In [65]:
from sklearn.metrics import f1_score
print(f1_score(ee["label"],ee["pred"],average="macro"))

0.5906113663783629


In [19]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [23]:
submission = pd.read_csv("../input/data/eval/info.csv")
image_paths = [os.path.join("../input/data/eval/images",img_id) for img_id in submission.ImageID]

test_data = TestDataset(image_paths,transform)
test_dataloader = torch.utils.data.DataLoader(test_data,batch_size=128, shuffle=False)

all_predictions = []
for images in test_dataloader:
    with torch.no_grad():
        images = images.to(device)
        pred = model_ft(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

In [24]:
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,2
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,16
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,2
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [111]:
torch.cuda.empty_cache()